In [17]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import tensorflow as tf
import xml.etree.ElementTree as ET
import glob
import shutil

In [3]:
def find_first_n_files_in_folder(folder_path, n=5):
    files = []
    for root, dirs, file_names in os.walk(folder_path):
        for file_name in file_names:
            files.append(os.path.join(root, file_name))
            if len(files) >= n:
                return files
    return files

# Example usage
folder_path = '/kaggle/input/rdd2022-india/India/test/images'
first_five_files = find_first_n_files_in_folder(folder_path)
for file in first_five_files:
    print(file)

/kaggle/input/rdd2022-india/India/test/images/India_000610.jpg
/kaggle/input/rdd2022-india/India/test/images/India_002833.jpg
/kaggle/input/rdd2022-india/India/test/images/India_002185.jpg
/kaggle/input/rdd2022-india/India/test/images/India_005060.jpg
/kaggle/input/rdd2022-india/India/test/images/India_004591.jpg


In [4]:
def parse_xml_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    data = {
        'filename': root.find('filename').text,
        'width': int(root.find('size/width').text),
        'height': int(root.find('size/height').text),
    }

    objects = []
    for obj in root.findall('object'):
        if obj.find('name').text in ['D40','D20','D00','D44']:
#     for obj in root.findall('object'):
            obj_data = {
                'class': obj.find('name').text,
                'xmin': int(obj.find('bndbox/xmin').text),
                'ymin': int(obj.find('bndbox/ymin').text),
                'xmax': int(obj.find('bndbox/xmax').text),
                'ymax': int(obj.find('bndbox/ymax').text),
            }
            objects.append(obj_data)
    
    data['objects'] = objects
    return data

def parse_annotations(annotation_dir):
    annotations = []
    for xml_file in tqdm(os.listdir(annotation_dir)):
        if xml_file.endswith('.xml'):
            annotation = parse_xml_annotation(os.path.join(annotation_dir, xml_file))
            annotations.append(annotation)
    return annotations

# Example usage
annotation_dir = '/kaggle/input/rdd2022-india/India/train/annotations/xmls'
annotations = parse_annotations(annotation_dir)


  0%|          | 0/7706 [00:00<?, ?it/s]

In [5]:
annotations[0:5]

[{'filename': 'India_007360.jpg', 'width': 720, 'height': 720, 'objects': []},
 {'filename': 'India_006920.jpg',
  'width': 720,
  'height': 720,
  'objects': [{'class': 'D40',
    'xmin': 551,
    'ymin': 619,
    'xmax': 584,
    'ymax': 647},
   {'class': 'D40', 'xmin': 380, 'ymin': 573, 'xmax': 413, 'ymax': 601},
   {'class': 'D40', 'xmin': 216, 'ymin': 628, 'xmax': 328, 'ymax': 687}]},
 {'filename': 'India_000805.jpg', 'width': 720, 'height': 720, 'objects': []},
 {'filename': 'India_000865.jpg',
  'width': 720,
  'height': 720,
  'objects': [{'class': 'D44',
    'xmin': 272,
    'ymin': 528,
    'xmax': 345,
    'ymax': 580}]},
 {'filename': 'India_004167.jpg', 'width': 720, 'height': 720, 'objects': []}]

In [6]:
len(annotations)

7706

In [7]:
def annotations_to_dataframe(annotations):
    rows = []
    for annotation in annotations:
        filename = annotation['filename']
        width = annotation['width']
        height = annotation['height']
        if not annotation['objects']:
            row = {
                'filename': filename,
                'width': width,
                'height': height,
                'class': None,  # No object class
                'xmin': None,   # No bounding box
                'ymin': None,
                'xmax': None,
                'ymax': None
            }
            rows.append(row)
        else:
            for obj in annotation['objects']:
                row = {
                    'filename': filename,
                    'width': width,
                    'height': height,
                    'class': obj['class'],
                    'xmin': obj['xmin'],
                    'ymin': obj['ymin'],
                    'xmax': obj['xmax'],
                    'ymax': obj['ymax']
                }
                rows.append(row)
    return pd.DataFrame(rows)

# Example usage
df = annotations_to_dataframe(annotations)
df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,India_007360.jpg,720,720,None,NaN,NaN,NaN,NaN
1,India_006920.jpg,720,720,D40,551.0,619.0,584.0,647.0
2,India_006920.jpg,720,720,D40,380.0,573.0,413.0,601.0
3,India_006920.jpg,720,720,D40,216.0,628.0,328.0,687.0
4,India_000805.jpg,720,720,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
11880,India_009238.jpg,720,720,None,NaN,NaN,NaN,NaN
11881,India_007572.jpg,720,720,D40,1.0,421.0,312.0,716.0
11882,India_007572.jpg,720,720,D40,348.0,442.0,447.0,531.0
11883,India_000372.jpg,720,720,None,NaN,NaN,NaN,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11885 entries, 0 to 11884
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   filename  11885 non-null  object 
 1   width     11885 non-null  int64  
 2   height    11885 non-null  int64  
 3   class     7825 non-null   object 
 4   xmin      7825 non-null   float64
 5   ymin      7825 non-null   float64
 6   xmax      7825 non-null   float64
 7   ymax      7825 non-null   float64
dtypes: float64(4), int64(2), object(2)
memory usage: 742.9+ KB


In [9]:
df.to_csv('annotations.csv')

In [10]:
df.isna().sum()

filename       0
width          0
height         0
class       4060
xmin        4060
ymin        4060
xmax        4060
ymax        4060
dtype: int64

In [11]:
final_df =df.copy()
final_df.dropna(inplace=True)
final_df.reset_index(inplace=True)
final_df.drop('index',axis=1,inplace=True)
final_df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,India_006920.jpg,720,720,D40,551.0,619.0,584.0,647.0
1,India_006920.jpg,720,720,D40,380.0,573.0,413.0,601.0
2,India_006920.jpg,720,720,D40,216.0,628.0,328.0,687.0
3,India_000865.jpg,720,720,D44,272.0,528.0,345.0,580.0
4,India_009855.jpg,720,720,D44,466.0,663.0,562.0,718.0
...,...,...,...,...,...,...,...,...
7820,India_002152.jpg,720,720,D44,276.0,501.0,325.0,555.0
7821,India_009075.jpg,720,720,D20,118.0,486.0,378.0,669.0
7822,India_009075.jpg,720,720,D20,530.0,487.0,695.0,659.0
7823,India_007572.jpg,720,720,D40,1.0,421.0,312.0,716.0


In [12]:
base_path = '/kaggle/input/rdd2022-india/India/train/images/'
final_df['image_path'] = final_df['filename'].apply(lambda x: os.path.join(base_path, x))
final_df

,filename,width,height,class,xmin,ymin,xmax,ymax,image_path
0,India_006920.jpg,720,720,D40,551.0,619.0,584.0,647.0,/kaggle/input/rdd2022-india/India/train/images...
1,India_006920.jpg,720,720,D40,380.0,573.0,413.0,601.0,/kaggle/input/rdd2022-india/India/train/images...
2,India_006920.jpg,720,720,D40,216.0,628.0,328.0,687.0,/kaggle/input/rdd2022-india/India/train/images...
3,India_000865.jpg,720,720,D44,272.0,528.0,345.0,580.0,/kaggle/input/rdd2022-india/India/train/images...
4,India_009855.jpg,720,720,D44,466.0,663.0,562.0,718.0,/kaggle/input/rdd2022-india/India/train/images...
...,...,...,...,...,...,...,...,...,...
7820,India_002152.jpg,720,720,D44,276.0,501.0,325.0,555.0,/kaggle/input/rdd2022-india/India/train/images...
7821,India_009075.jpg,720,720,D20,118.0,486.0,378.0,669.0,/kaggle/input/rdd2022-india/India/train/images...
7822,India_009075.jpg,720,720,D20,530.0,487.0,695.0,659.0,/kaggle/input/rdd2022-india/India/train/images...
7823,India_007572.jpg,720,720,D40,1.0,421.0,312.0,716.0,/kaggle/input/rdd2022-india/India/train/images...


In [13]:
final_df['image_path'][0]

'/kaggle/input/rdd2022-india/India/train/images/India_006920.jpg'

In [14]:
final_df['class'].unique()

array(['D40', 'D44', 'D20', 'D00'], dtype=object)

In [15]:
class_mapping = {
    'D40': 0,
    'D44': 1,
    'D00': 2,
    'D20': 3
}

output_dir = '/kaggle/working/all_labels'
os.makedirs(output_dir, exist_ok=True)

def convert_to_yolo_format(row):
    if pd.isna(row['class']):
        return

    class_id = class_mapping[row['class']]
    x_center = (row['xmin'] + row['xmax']) / 2.0 / row['width']
    y_center = (row['ymin'] + row['ymax']) / 2.0 / row['height']
    bbox_width = (row['xmax'] - row['xmin']) / row['width']
    bbox_height = (row['ymax'] - row['ymin']) / row['height']

    return f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}"

def write_yolo_annotation(final_df, output_dir):
    grouped = final_df.groupby('filename')
    for filename, group in grouped:
        yolo_annotations = group.apply(convert_to_yolo_format, axis=1).dropna().tolist()
        label_filename = os.path.join(output_dir, filename.replace('.jpg','.txt'))
        with open(label_filename, 'w') as f:
            f.write("\n".join(yolo_annotations))

write_yolo_annotation(final_df, output_dir)

In [21]:
os.makedirs('/kaggle/working/datasets/labels/train/', exist_ok=True)
os.makedirs('/kaggle/working/datasets/labels/val/', exist_ok=True)
os.makedirs('/kaggle/working/datasets/images/train/', exist_ok=True)
os.makedirs('/kaggle/working/datasets/images/val/', exist_ok=True)

In [29]:
label_source_dir = '/kaggle/working/all_labels'
label_train_dir = '/kaggle/working/datasets/labels/train/'
label_val_dir = '/kaggle/working/datasets/labels/val/'

image_source_dir ='/kaggle/input/rdd2022-india/India/train/images'
image_train_dir = '/kaggle/working/datasets/images/train/'
image_val_dir = '/kaggle/working/datasets/images/val/'

image_files = final_df['image_path'].unique()
train_files = image_files[:int(0.8*len(image_files))]
val_files = image_files[int(0.8*len(image_files)):]

train_images = [os.path.splitext(os.path.basename(path.strip()))[0] + ".jpg" for path in train_files]
val_images = [os.path.splitext(os.path.basename(path.strip()))[0] + ".jpg" for path in val_files]
train_labels = [os.path.splitext(os.path.basename(path.strip()))[0] + ".txt" for path in train_files]
val_labels = [os.path.splitext(os.path.basename(path.strip()))[0] + ".txt" for path in val_files]


for filename in train_images:
    if filename.endswith('.jpg'):  # Ensure we're only moving label files
        shutil.copy(os.path.join(image_source_dir, filename), os.path.join(image_train_dir, filename))
for filename in val_images:
    if filename.endswith('.jpg'):  # Ensure we're only moving label files
        shutil.copy(os.path.join(image_source_dir, filename), os.path.join(image_val_dir, filename))

for filename in train_labels:
    if filename.endswith('.txt'):  # Ensure we're only moving label files
        shutil.move(os.path.join(label_source_dir, filename), os.path.join(label_train_dir, filename))
for filename in val_labels:
    if filename.endswith('.txt'):  # Ensure we're only moving label files
        shutil.move(os.path.join(label_source_dir, filename), os.path.join(label_val_dir, filename))

        

# with open('/kaggle/working/dataset/images/train', 'w') as f:
#     for item in train_files:
#         f.write("%s\n" % item)
        
        

# with open('/kaggle/working/val.txt', 'w') as f:
#     for item in val_files:
#         f.write("%s\n" % item)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/all_labels/India_006920.txt'

In [24]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Cloning into 'yolov5'...
remote: Enumerating objects: 16836, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 16836 (delta 1), reused 6 (delta 0), pack-reused 16825
Receiving objects: 100% (16836/16836), 15.57 MiB | 32.14 MiB/s, done.
Resolving deltas: 100% (11550/11550), done.
/kaggle/working/yolov5


In [25]:
# Create the dataset.yaml file
dataset_yaml = """
train: /kaggle/working/datasets/images/train
val: /kaggle/working/datasets/images/val

# Number of classes
nc: 4

# Class names
names: [0, 1, 2, 3]
"""

# Save the dataset.yaml file
with open('/kaggle/working/dataset.yaml', 'w') as f:
    f.write(dataset_yaml)

In [26]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 45.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.6/862.6 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 79.2 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.0.3
    Uninstalling setuptools-69.0.3:
      Successfully uninstalled setuptools-69.0.3
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.5.0
    Uninstalling Pillow-9.5.0:
      Successfully uninstalled Pillow-9.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
conda 24.5.0 requires packaging>=23.0, but you have packaging 21.3 which is incompatible.
jupyterlab

In [27]:
!wandb off

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.


In [31]:
!python train.py --img 640 --batch 16 --epochs 50 --data /kaggle/working/dataset.yaml --cfg yolov5s.yaml --weights yolov5s.pt --name road_damage_detection

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-07-31 06:40:31.018727: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 06:40:31.018784: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 06:40:31.020330: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=yolov5s.yaml, data=/kaggle/working/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False

In [41]:
# cache_path = '/kaggle/working/'
# if os.path.exists(cache_path):
#     os.remove(cache_path)
#     print(f"The cache file '{cache_path}' has been deleted.")
# else:
#     print(f"The cache file '{cache_path}' does not exist.")

IsADirectoryError: [Errno 21] Is a directory: '/kaggle/working/'